In [19]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', None)

df=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
df_result=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv")   

df['bid_ask_spread']=df['ask']-df['bid']
df['trade']=df['trade'].astype(int)

value_categor=['venue','action','side']

for k in value_categor:

    df_encoded = pd.get_dummies(df[k], prefix=k).astype(int)
    df = df.merge( df_encoded , left_index = True , right_index = True)
    df.drop( columns=[k] , inplace = True )

df.drop(columns=['obs_id','order_id'],inplace=True)

value_no_center=['price' , 'bid' , 'ask' , 'bid_size', 'ask_size', 'bid_ask_spread' ]

for k in value_no_center:
    
    df[k] = df[k] / max( df[k] )

In [20]:
X = df.values.reshape(160800, 100, 19)
y = df_result['eqt_code_cat'].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Hypothèse : Les données sont dans les variables x_train et y_train, prétraitées selon la description.

# Définition du modèle
def create_benchmark_model():
    # Entrée : un tenseur de taille (100, 30)
    inputs = layers.Input(shape=(100, 19))

    # Deux cellules GRU : une en avant, une en arrière
    gru_fwd = layers.GRU(64, return_sequences=False)(inputs)
    gru_bwd = layers.GRU(64, return_sequences=False, go_backwards=True)(inputs)

    # Concaténation des sorties
    concatenated = layers.concatenate([gru_fwd, gru_bwd])

    # Couche dense de 128 vers 64 dimensions avec activation SeLU
    dense_1 = layers.Dense(64, activation='selu')(concatenated)

    # Couche dense de 64 vers 24 (softmax pour les probabilités des classes)
    output = layers.Dense(24, activation='softmax')(dense_1)

    # Création du modèle
    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    # Compilation du modèle avec Adam et une fonction de perte cross-entropy
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-3),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Création du modèle
model = create_benchmark_model()

# Hypothèse : x_train et y_train sont prétraités et disponibles
history = model.fit(x_train, y_train, batch_size = 1000, epochs = 20)

# Évaluation
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 33s 249ms/step - accuracy: 0.0772 - loss: 3.0964
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 41s 315ms/step - accuracy: 0.1636 - loss: 2.6249
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 41s 317ms/step - accuracy: 0.2193 - loss: 2.4314
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 41s 319ms/step - accuracy: 0.2688 - loss: 2.2756
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 39s 301ms/step - accuracy: 0.3029 - loss: 2.1624
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 38s 296ms/step - accuracy: 0.3286 - loss: 2.0821
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 39s 303ms/step - accuracy: 0.3512 - loss: 2.0143
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 41s 317ms/step - accuracy: 0.3745 - loss: 1.9431
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 42s 329ms/step - accuracy: 0.3886 - loss: 1.8923
Epoch 10/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 42s 329ms/step - accuracy: 0.4025 - loss: 1.8474
Epoch 11/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 48s 370ms/step - accuracy: 0.4088 - loss: 1.8253
Epoch 12/20
129/129